In [1]:
import numpy as np
import random

from mahjong.shanten import Shanten
from mahjong.agari import Agari
from mahjong.hand_calculating.hand import HandCalculator
from mahjong.hand_calculating.hand_config import HandConfig
from mahjong.tile import TilesConverter

SHANTEN = Shanten()
CALCULATOR = HandCalculator()
AGARI = Agari()
NUM_HAIS = 34

In [2]:
# load data
def load_hand(files):
    hands = []
    for file_path in files:
        for line in open(file_path, "r"):
            fields = line[:-1].split(":")
            shanten = int(fields[0])
            hand = [int(hid) for hid in fields[1].split(",")]
            hands.append((shanten, hand))
    return hands

In [3]:
hands = load_hand(["/Users/xinran.he/GitProjects/mahjong/data/single_hand_efficiency/20180101.txt"])

In [4]:
print hands[0]

(4, [0, 1, 2, 5, 6, 6, 12, 15, 16, 16, 18, 23, 29])


In [5]:
def get_total_score(hand34, wintile34):
    result = CALCULATOR.estimate_hand_value(TilesConverter.to_136_array(hand34), wintile34 * 4, config=HandConfig(is_tsumo=True))
    return result.cost['main'] + 2 * result.cost['additional']

In [6]:
def is_agari(hand34):
    return AGARI.is_agari(hand34)

In [7]:
def greedy_shanten(hand34):
    initial_shanten = SHANTEN.calculate_shanten(hand34)
    discards = []
    for i in xrange(34):
        if hand34[i] > 0:
            hand34[i] -= 1
            if SHANTEN.calculate_shanten(hand34) == initial_shanten:
                discards.append(i)
            hand34[i] += 1
    return random.choice(discards)

In [8]:
def compute_jinzhang(hand34, left_tiles):
    initial_shanten = SHANTEN.calculate_shanten(hand34)
    result = np.zeros(NUM_HAIS)
    for i in xrange(NUM_HAIS):
        num_hais = left_tiles[i]
        if num_hais > 0:
            hand34[i] += 1
            new_shanten = SHANTEN.calculate_shanten(hand34)
            if (new_shanten < initial_shanten):
                result[i] = num_hais
            hand34[i] -= 1
    return np.sum(result)

In [30]:
def compute_discard_jinzhange(tiles, left_tiles):
    initial_shanten = SHANTEN.calculate_shanten(tiles)
    results = []
    best_jinzhang_num = 0
    for i in xrange(NUM_HAIS):
        if tiles[i] > 0:
            tiles[i] -= 1
            new_shanten = SHANTEN.calculate_shanten(tiles)
            if new_shanten == initial_shanten:
                num_jinzhang = compute_jinzhang(tiles, left_tiles)
                if num_jinzhang > best_jinzhang_num:
                    best_jinzhang_num = num_jinzhang
                results.append((num_jinzhang, i))
            tiles[i] += 1
    discards = [r[1] for r in results if r[0] == best_jinzhang_num]
    return random.choice(discards)

In [10]:
def discard(tiles):
    return tiles[-1]

In [11]:
TO_GRAPH_LIST = [
    "🀇", "🀈", "🀉", "🀊", "🀋", "🀌", "🀍", "🀎", "🀏", "🀙", "🀚", "🀛", "🀜", "🀝", "🀞", "🀟", "🀠", "🀡",
    "🀐", "🀑", "🀒", "🀓", "🀔", "🀕", "🀖", "🀗", "🀘", "🀀", "🀁", "🀂", "🀃", "🀆", "🀅", "🀄"
]

In [12]:
def tiles34_to_list(tiles):
    result = []
    for i in xrange(34):
        for j in xrange(tiles[i]):
            result.append(i)
    return sorted(result)

In [13]:
def print_tile34_hand(tiles):
    result = ""
    for i in xrange(NUM_HAIS):
        for j in xrange(tiles[i]):
            result += TO_GRAPH_LIST[i]
    print result

In [41]:
MAX_ROUND = 30
NEGATIVE_REWARD = 0
DISCOUNT_FACTOR = 0.95

def simulate(init_hand, strategy, is_debug=False):
    # init
    current_hand = [0] * NUM_HAIS
    left_tiles = [4] * NUM_HAIS
    for hai in init_hand:
        left_tiles[hai] -= 1
        current_hand[hai] += 1
    yama = tiles34_to_list(left_tiles)
    random.shuffle(yama)
    
    rewards = [-1] * MAX_ROUND
    # (hand, reward)
    data = []
    for i in xrange(MAX_ROUND):
        data.append([tiles34_to_list(current_hand), NEGATIVE_REWARD])
        
        # draw new tile
        new_tile = yama[i]
        
        if is_debug:
            print_tile34_hand(current_hand)
            print TO_GRAPH_LIST[new_tile]
            
        current_hand[new_tile] += 1
        left_tiles[new_tile] -= 1
        if is_agari(current_hand):
            data[i][1] = get_total_score(current_hand, new_tile) / 100.0
            if is_debug:
                print "Agali"
                print_tile34_hand(current_hand)
                print data[i][1]
                print "============"
            break
        else:
            discard = compute_discard_jinzhange(current_hand, left_tiles)
            current_hand[discard] -= 1
            if is_debug:
                print TO_GRAPH_LIST[discard]
    
    # generate reward
    i -= 1
    while i >= 0:
        data[i][1] += data[i + 1][1] * DISCOUNT_FACTOR
        i -= 1
    
    return data 

In [42]:
data = simulate(hands[0][1], discard)
print data

[[[0, 1, 2, 5, 6, 6, 12, 15, 16, 16, 18, 23, 29], 13.017892666607663], [[0, 1, 2, 5, 6, 6, 12, 14, 15, 16, 16, 18, 23], 13.703044912218592], [[0, 1, 2, 5, 6, 6, 12, 14, 15, 15, 16, 16, 23], 14.424257802335362], [[0, 1, 2, 5, 6, 6, 14, 15, 15, 16, 16, 21, 23], 15.183429265616171], [[0, 1, 2, 5, 6, 6, 14, 15, 15, 16, 16, 23, 23], 15.982557121701234], [[0, 1, 2, 5, 6, 14, 15, 15, 16, 16, 16, 23, 23], 16.823744338632878], [[0, 1, 2, 5, 6, 14, 15, 15, 16, 16, 16, 23, 23], 17.709204566981978], [[0, 1, 2, 5, 6, 14, 15, 15, 16, 16, 16, 23, 23], 18.641267965244186], [[1, 2, 3, 5, 6, 14, 15, 15, 16, 16, 16, 23, 23], 19.622387331835984], [[1, 2, 3, 5, 6, 7, 14, 15, 16, 16, 16, 23, 23], 20.655144559827352], [[1, 2, 3, 5, 6, 7, 14, 15, 16, 16, 16, 23, 23], 21.742257431397213], [[1, 2, 3, 5, 6, 7, 14, 15, 16, 16, 16, 23, 23], 22.886586769891803], [[1, 2, 3, 5, 6, 7, 14, 15, 16, 16, 16, 23, 23], 24.09114396830716], [[1, 2, 3, 5, 6, 7, 14, 15, 16, 16, 16, 23, 23], 25.35909891400754], [[1, 2, 3, 5, 6, 

In [ ]:
from mahjong.tile import TilesConverter
tiles = TilesConverter.string_to_34_array(man='', 
                                          pin='234445789', 
                                          sou='1568')

In [33]:
print data

[[[0, 1, 2, 5, 6, 6, 12, 15, 16, 16, 18, 23, 29], 7.0], [[1, 2, 2, 5, 6, 6, 12, 15, 16, 16, 18, 23, 29], 8.0], [[1, 2, 2, 5, 6, 6, 11, 12, 15, 16, 16, 18, 29], 9.0], [[1, 2, 2, 5, 6, 6, 11, 12, 15, 16, 16, 18, 29], 10.0], [[1, 2, 2, 5, 6, 6, 11, 12, 15, 16, 16, 18, 29], 11.0], [[1, 2, 2, 6, 6, 11, 11, 12, 15, 16, 16, 18, 29], 12.0], [[2, 2, 6, 6, 11, 11, 12, 15, 16, 16, 18, 29, 29], 13.0], [[2, 2, 6, 6, 10, 11, 11, 12, 16, 16, 18, 29, 29], 14.0], [[2, 2, 6, 6, 10, 11, 11, 13, 16, 16, 18, 29, 29], 15.0], [[2, 2, 6, 6, 10, 11, 11, 13, 16, 16, 26, 29, 29], 16.0], [[2, 2, 6, 6, 10, 11, 11, 13, 16, 16, 26, 29, 29], 17.0], [[2, 2, 6, 6, 10, 11, 11, 13, 16, 16, 26, 29, 29], 18.0], [[2, 2, 6, 6, 10, 11, 11, 13, 16, 16, 26, 29, 29], 19.0], [[2, 2, 6, 6, 10, 11, 11, 13, 16, 16, 26, 29, 29], 20.0], [[2, 2, 6, 6, 11, 11, 13, 16, 16, 26, 27, 29, 29], 21.0], [[2, 2, 6, 6, 11, 11, 13, 16, 16, 26, 27, 29, 29], 22.0], [[2, 2, 6, 6, 9, 11, 11, 13, 16, 16, 27, 29, 29], 23.0], [[2, 2, 4, 6, 6, 9, 11, 11, 

In [ ]:
print tiles

In [ ]:
win_tile = TilesConverter.string_to_136_array(sou='4')[0]
print win_tile